In [6]:
import * as React from "react";
import { useState, useEffect, useCallback } from "react";
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  shorthands,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";

const useStyles = makeStyles({
  container: {
    display: "flex",
    flexDirection: "column",
    width: "100%",
    height: "100vh",
    backgroundColor: tokens.colorNeutralBackground3,
    ...shorthands.padding("24px"),
    boxSizing: "border-box",
    overflow: "hidden",
  },
  actionBar: {
    display: "flex",
    flexDirection: "column",
    width: "100%",
    backgroundColor: tokens.colorNeutralBackground2,
    borderRadius: tokens.borderRadiusMedium,
    transition: "all 0.2s ease",
    boxShadow: tokens.shadow4,
    ":hover": {
      backgroundColor: tokens.colorNeutralBackground2Hover,
      transform: "translateY(-2px)",
      boxShadow: tokens.shadow8,
    },
  },
  actionContent: {
    display: "flex",
    flexDirection: "column",
    gap: "8px",
    padding: "16px",
  },
  actionTitle: {
    fontSize: tokens.fontSizeBase500,
    fontWeight: tokens.fontWeightSemibold,
    color: tokens.colorNeutralForeground1,
  },
  actionDescription: {
    fontSize: tokens.fontSizeBase300,
    color: tokens.colorNeutralForeground3,
  },
  iconWrapper: {
    display: "flex",
    alignItems: "center",
    justifyContent: "center",
    width: "32px",
    height: "32px",
    borderRadius: "50%",
    backgroundColor: "#106ebe",
    color: tokens.colorNeutralBackground1,
    marginRight: "12px",
  },
  input: {
    width: "100%",
    minHeight: "100px",
    resize: "vertical",
    backgroundColor: "transparent",
    border: `1px solid ${tokens.colorNeutralStroke1}`,
    borderRadius: tokens.borderRadiusMedium,
    padding: "8px",
    color: tokens.colorNeutralForeground1,
    ":focus": {
      outlineColor: tokens.colorBrandStroke1Focus,
    },
  },
  buttonContainer: {
    display: "flex",
    flexWrap: "wrap",
    gap: "8px",
    marginTop: "16px",
  },
  statusContainer: {
    display: "flex",
    justifyContent: "center",
    alignItems: "center",
    marginTop: "24px",
    minHeight: "60px",
  },
  statusMessage: {
    padding: "12px 24px",
    borderRadius: tokens.borderRadiusMedium,
    backgroundColor: tokens.colorNeutralBackground1,
    color: tokens.colorNeutralForeground1,
    boxShadow: tokens.shadow4,
    textAlign: "center",
    fontSize: tokens.fontSizeBase300,
    fontWeight: tokens.fontWeightSemibold,
    display: "flex",
    flexDirection: "column",
    alignItems: "center",
    justifyContent: "center",
    maxWidth: "80%",
  },
  successMessage: {
    backgroundColor: tokens.colorSuccessBackground,
    color: tokens.colorSuccessForeground1,
  },
  errorMessage: {
    backgroundColor: tokens.colorErrorBackground,
    color: tokens.colorErrorForeground1,
  },
  responseContainer: {
    marginTop: "24px",
    padding: "16px",
    backgroundColor: tokens.colorNeutralBackground1,
    borderRadius: tokens.borderRadiusMedium,
    boxShadow: tokens.shadow4,
    overflowY: "auto",
    maxHeight: "calc(100vh - 400px)", // Adjust as needed
  },
  responseText: {
    whiteSpace: "pre-wrap",
    wordBreak: "break-word",
  },
});

const EmailGenerator = ({ userId = "user1" }) => {
  const styles = useStyles();
  const [isLoading, setIsLoading] = useState(true);
  const [isProcessing, setIsProcessing] = useState(false);
  const [userConfig, setUserConfig] = useState(null);
  const [statusMessage, setStatusMessage] = useState(null);
  const [error, setError] = useState(null);
  const [inputValue, setInputValue] = useState("");
  const [generatedResponse, setGeneratedResponse] = useState("");
  const [selectedAction, setSelectedAction] = useState(null);

  const fetchUserConfig = useCallback(async () => {
    setIsLoading(true);
    setError(null);
    try {
      console.log(`Fetching user configuration for user: ${userId}`);
      const response = await fetch(`http://localhost:8001/getUserConfig/${userId}`);
      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const data = await response.json();
      console.log("Received user configuration:", data);
      setUserConfig(data);
    } catch (e) {
      console.error("Error fetching user configuration:", e);
      setError(`Failed to load user configuration: ${e.message}`);
    } finally {
      setIsLoading(false);
    }
  }, [userId]);

  useEffect(() => {
    fetchUserConfig();
  }, [fetchUserConfig]);

  const getEmailContent = async () => {
    return new Promise((resolve, reject) => {
      Office.context.mailbox.item.body.getAsync(Office.CoercionType.Text, (result) => {
        if (result.status === Office.AsyncResultStatus.Succeeded) {
          resolve(result.value);
        } else {
          reject(new Error(result.error.message));
        }
      });
    });
  };

  const handleButtonClick = (action) => {
    console.log(`Button clicked: ${action.label}, Requires input: ${action.requiresInput}`);
    if (action.requiresInput) {
      console.log("Setting selected action for input");
      setSelectedAction(action);
    } else {
      console.log("Handling action directly");
      handleAction(action);
    }
  };

  const handleAction = async (action, inputValue = null) => {
    setIsProcessing(true);
    setError(null);
    setStatusMessage(null);
    setGeneratedResponse("");
    try {
      console.log(`Handling action: ${action.label}, Input value: ${inputValue}`);
      const content = await getEmailContent();
      const payload = {
        userId: userConfig.userId,
        emailContent: content,
        ...(inputValue && { prompt: inputValue }),
      };

      console.log(`Sending request to: ${action.apiEndpoint}`);
      const response = await fetch(action.apiEndpoint, {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify(payload),
      });

      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const responseData = await response.json();

      console.log("Received response:", responseData);
      setGeneratedResponse(responseData.generatedContent);

      setStatusMessage({
        text: `${action.label} completed successfully!`,
      });
      setInputValue("");
      setSelectedAction(null);
    } catch (e) {
      console.error(`Error in handleAction: ${e.message}`);
      setError(`Failed to ${action.label.toLowerCase()}. Please try again.`);
    } finally {
      setIsProcessing(false);
    }
  };

  if (isLoading) {
    return (
      <div className={styles.container}>
        <div className={styles.statusContainer}>
          <Spinner size="large" label="Loading user data..." />
        </div>
      </div>
    );
  }

  if (error) {
    return (
      <div className={styles.container}>
        <div className={styles.statusContainer}>
          <div className={`${styles.statusMessage} ${styles.errorMessage}`}>
            {error}
          </div>
        </div>
      </div>
    );
  }

  return (
    <div className={styles.container}>
      <div className={styles.actionBar}>
        <div className={styles.actionContent}>
          <div style={{ display: "flex", alignItems: "center" }}>
            <div className={styles.iconWrapper}>
              <FluentIcons.MailTemplate24Regular />
            </div>
            <Text className={styles.actionTitle}>Email Actions</Text>
          </div>
          <Text className={styles.actionDescription}>Select an action to perform on your email.</Text>
          {selectedAction && selectedAction.requiresInput && (
            <>
              <Text className={styles.actionDescription}>{selectedAction.description}</Text>
              <textarea
                className={styles.input}
                placeholder="Enter any additional instructions here..."
                value={inputValue}
                onChange={(e) => setInputValue(e.target.value)}
              />
            </>
          )}
          <div className={styles.buttonContainer}>
            {userConfig.buttons.map((action, index) => (
              <Button
                key={index}
                appearance={selectedAction === action ? "primary" : "secondary"}
                onClick={() => handleButtonClick(action)}
                disabled={isProcessing || (selectedAction && selectedAction !== action)}
              >
                {action.label}
              </Button>
            ))}
            {selectedAction && selectedAction.requiresInput && (
              <Button
                appearance="primary"
                onClick={() => handleAction(selectedAction, inputValue)}
                disabled={isProcessing || !inputValue.trim()}
              >
                Submit
              </Button>
            )}
          </div>
        </div>
      </div>
      <div className={styles.statusContainer}>
        {isProcessing && (
          <div className={styles.statusMessage}>
            <Spinner size="tiny" />
            <span style={{ marginLeft: '8px' }}>Processing your request...</span>
          </div>
        )}
        {statusMessage && (
          <div className={`${styles.statusMessage} ${styles.successMessage}`}>
            <div>{statusMessage.text}</div>
          </div>
        )}
        {error && (
          <div className={`${styles.statusMessage} ${styles.errorMessage}`}>
            {error}
          </div>
        )}
      </div>
      {generatedResponse && (
        <div className={styles.responseContainer}>
          <Text className={styles.responseText}>{generatedResponse}</Text>
        </div>
      )}
    </div>
  );
};

export default EmailGenerator;

ModuleNotFoundError: No module named 'streamlit'